# Parkeervakken in Amsterdam

[![](../images/Logo-Gemeente-Amsterdam.png)](https://maps.amsterdam.nl/open_geodata/)

In [1]:
import sys, os
print(f'python     {sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}')

import pandas as pd
print(f'pandas     {pd.__version__}')

import geopandas as gpd
print(f'geopandas  {gpd.__version__}')

from shapely.geometry import Point, Polygon, MultiPolygon
print(f'shapely    {sys.modules["shapely"].__version__}')

import matplotlib.pyplot as plt
print(f'matplotlib {sys.modules["matplotlib"].__version__}')

import folium
print(f'folium     {folium.__version__}')

python     3.6.6
pandas     0.23.4
geopandas  0.4.0
shapely    1.6.4.post2
matplotlib 2.2.2
folium     0.7.0


## Parkeervakken

In [2]:
# lees data
parkeervakken = gpd.read_file('../shapefiles/alle_parkeervakken.shp')
parkeervakken.drop(['id', 'geo_id', 'straatnaam', 'soort', 'type', 'e_type', 'bord'], axis=1, inplace=True)
parkeervakken.dropna(inplace=True)

In [3]:
parkeervakken.head()

,buurtcode,stadsdeel,aantal,geometry
0,B10g,westpoort_nf,1,"POLYGON ((111725.48 491372.13, 111724.98 49136..."
1,B10g,westpoort_nf,1,"POLYGON ((111725.98 491374.5, 111725.48 491372..."
2,B10g,westpoort_nf,1,"POLYGON ((111726.49 491376.87, 111725.98 49137..."
3,B10g,westpoort_nf,1,"POLYGON ((111726.99 491379.23, 111726.49 49137..."
4,B10g,westpoort,1,"POLYGON ((111727.49 491381.6, 111726.99 491379..."


## 1. Buurten

In [4]:
# lees data
buurt = gpd.read_file('../shapefiles/brtk2010_ind2005_region.shp')
buurt.drop(['BUURT', 'BC', 'LOKATIE', 'OPP', 'REGIEBUURT', 'REGIEBUU0', 'VEILIGHEID'], axis=1, inplace=True)

In [5]:
# aantal hectare
buurt['HA'] = buurt.geometry.area / 100**2

In [6]:
# aantal parkeervakken
buurt_aantal = parkeervakken[['buurtcode', 'aantal']].groupby(by='buurtcode').sum()

In [7]:
# merge data
geodata_buurt = pd.merge(buurt, buurt_aantal, how='left', left_on='SDBRT', right_on='buurtcode').fillna(0)

In [8]:
# aantal parkeervakken per hectare
geodata_buurt['aantal_HA'] = geodata_buurt.aantal / geodata_buurt.HA

### 1. 1 Aantal parkeerplaatsen

In [9]:
m = folium.Map(location = [52.37, 4.95], zoom_start = 11)

folium.Choropleth(
    geo_data = geodata_buurt,
    name = 'geometry',
    data = geodata_buurt,
    columns = ['SDBRT', 'aantal'],
    key_on = 'feature.properties.SDBRT',
    fill_color = 'Reds',
    fill_opacity = 0.7,
    line_opacity = 0.8,
    legend_name = 'Aantal parkeervakken',
    reset = True).add_to(m)

m.save(os.path.join('..', 'output', 'buurt_aantal.html'))

m

### 1.2 Aantal parkeerplaatsen per KM$^2$

In [10]:
m = folium.Map(location = [52.37, 4.95], zoom_start = 11)

folium.Choropleth(
    geo_data = geodata_buurt,
    name = 'geometry',
    data = geodata_buurt,
    columns = ['SDBRT', 'aantal_HA'],
    key_on = 'feature.properties.SDBRT',
    fill_color = 'Greens',
    fill_opacity = 0.7,
    line_opacity = 0.8,
    legend_name = 'Aantal parkeervakken per hectare',
    reset = True).add_to(m)

m.save(os.path.join('..', 'output', 'buurt_aantal_hectare.html'))

m

## 2. Buurtcombinaties (wijken)

In [11]:
# lees data
buurtcombinatie = gpd.read_file('../shapefiles/bc2010zw_region.shp')
buurtcombinatie.drop(['WOONMILIEU', 'TYPE', 'OPP', 'RAYONCENTR'], axis=1, inplace=True)

In [12]:
# aantal hectare
buurtcombinatie['HA'] = buurtcombinatie.geometry.area / 100**2

In [13]:
# voeg kolom toe om te mergen
parkeervakken['SDBC'] = [x[:-1] for x in parkeervakken.buurtcode]

In [14]:
# aantal parkeervakken per buurtcombinatie
buurtcombinatie_aantal = parkeervakken[['SDBC', 'aantal']].groupby(by='SDBC').sum()

In [15]:
# merge
geodata_buurtcombinatie = pd.merge(buurtcombinatie, buurtcombinatie_aantal, how='left', on='SDBC').fillna(0)

In [16]:
# aantal parkeervakken per hectare
geodata_buurtcombinatie['aantal_HA'] = geodata_buurtcombinatie.aantal / geodata_buurtcombinatie.HA

### 2.1 Aantal parkeerplaatsen

In [17]:
m = folium.Map(location = [52.37, 4.95], zoom_start = 11)

folium.Choropleth(
    geo_data = geodata_buurtcombinatie,
    name = 'geometry',
    data = geodata_buurtcombinatie,
    columns = ['SDBC', 'aantal'],
    key_on = 'feature.properties.SDBC',
    fill_color = 'Reds',
    fill_opacity = 0.7,
    line_opacity = 0.8,
    legend_name = 'Aantal parkeervakken',
    reset = True).add_to(m)

m.save(os.path.join('..', 'output', 'buurtcombinatie_aantal.html'))

m

### 2.2 Aantal parkeerplaatsen per KM$^2$

In [18]:
m = folium.Map(location = [52.37, 4.95], zoom_start = 11)

folium.Choropleth(
    geo_data = geodata_buurtcombinatie,
    name = 'geometry',
    data = geodata_buurtcombinatie,
    columns = ['SDBC', 'aantal_HA'],
    key_on = 'feature.properties.SDBC',
    fill_color = 'Greens',
    fill_opacity = 0.7,
    line_opacity = 0.8,
    legend_name = 'Aantal parkeervakken per hectare',
    reset = True).add_to(m)

m.save(os.path.join('..', 'output', 'buurtcombinatie_aantal_hectare.html'))

m

## 3. Stadsdelen

In [19]:
# lees data
stadsdeel = gpd.read_file('../shapefiles/sd2010zw_region.shp')
stadsdeel.drop(['ZORG', 'OPP', 'EZGEBIED', 'SD09', 'SDNUMMER', 'PERCENTAGE'], axis=1, inplace=True)

In [20]:
# aantal hectare
stadsdeel['HA'] = stadsdeel.geometry.area / 100**2

In [21]:
# voeg kolom toe om te mergen
parkeervakken['SD'] = [x[0] for x in parkeervakken.buurtcode]

In [22]:
# aantal parkeervakken per buurtcombinatie
stadsdeel_aantal = parkeervakken[['SD', 'aantal']].groupby(by='SD').sum()

In [23]:
# merge
geodata_stadsdeel = pd.merge(stadsdeel, stadsdeel_aantal, how='left', on='SD').fillna(0)

In [24]:
# aantal parkeervakken hectare
geodata_stadsdeel['aantal_HA'] = geodata_stadsdeel.aantal / geodata_stadsdeel.HA

### 3.1 Aantal parkeerplaatsen

In [25]:
m = folium.Map(location = [52.37, 4.95], zoom_start = 11)

folium.Choropleth(
    geo_data = geodata_stadsdeel,
    name = 'geometry',
    data = geodata_stadsdeel,
    columns = ['SD', 'aantal'],
    key_on = 'feature.properties.SD',
    fill_color = 'Reds',
    fill_opacity = 0.7,
    line_opacity = 0.8,
    legend_name = 'Aantal parkeervakken',
    reset = True).add_to(m)

m.save(os.path.join('..', 'output', 'stadsdeel_aantal.html'))

m

### 3.2 Aantal parkeerplaatsen per KM$^2$

In [26]:
m = folium.Map(location = [52.37, 4.95], zoom_start = 11)

folium.Choropleth(
    geo_data = geodata_stadsdeel,
    name = 'geometry',
    data = geodata_stadsdeel,
    columns = ['SD', 'aantal_HA'],
    key_on = 'feature.properties.SD',
    fill_color = 'Greens',
    fill_opacity = 0.7,
    line_opacity = 0.8,
    legend_name = 'Aantal parkeervakken per heactare',
    reset = True).add_to(m)

m.save(os.path.join('..', 'output', 'stadsdeel_aantal_hectare.html'))

m